In [ ]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
"""
# If you're using Google Colab and not running locally, run this cell.

## Install dependencies
!apt-get install sox libsndfile1 ffmpeg
!pip install wget
!pip install text-unidecode

# ## Install NeMo
BRANCH = 'r1.22.0'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

## Grab the config we'll use in this example
!mkdir configs

# Adapters Support in NeMo Models

In NeMo, we often train models and fine-tune them for a specific task. This is a reasonable approach when the models are just a few million parameters. However, this approach quickly becomes infeasible when approaching hundreds of millions or even billions of parameters. 

As a potential solution to such a scenario, where fine-tuning a massive model is no longer feasible, we look to specialized [Adapters](https://arxiv.org/abs/1902.00751) to specialize our model on a specific domain or task. Adapters require a fraction of the total number of parameters as the original model and are much more efficient to fine-tune.

In this tutorial, we will discuss how to update any torch.nn.Module to support Adapters, and going further, how to enable NeMo models with Adapter support for their components.


## What are Adapters?

Adapters are a straightforward concept - one formulation can be shown by the diagram below. At their simplest, they are residual Feedforward layers that compress the input dimension ($D$) to a small bottleneck dimension ($H$), such that $R^D \text{->} R^H$, compute an activation (such as ReLU), finally mapping $R^H \text{->} R^D$ with another Feedforward layer. This output is then added to the input via a simple residual connection.

<div align="center">
  <img src="https://mermaid.ink/img/pako:eNptkLFqwzAQhl9F3ORAPDSjA4EUx6RgXEjbycpwWOdG1JaMfEoakrx7ZcfpUKrlxH_fz4d0gcoqggTqxp6qAzoW76k0Ipx1-WI6z3sRxyuRF1GOZ3KisK6d3YG8GFdZ9hRJeLbMDRmqvkRGpDLrTuiUiEWUigBtlyIVqzBnEqZ66I39dcX6iKytKXeUf-wn-286QoFeBMvmu0PTD-EfyXaQpP9JFmP_1XN4S3kfD8W4ue6o18pjc52gYQlzaMm1qFX4msuQSOADtSQhCdfaOupZgjS3QPpOIdNGabYOkhqbnuaAnu3b2VSQsPP0gFKNnw7bibr9AJkZdXU" height=100% />
</div>

-----

Adapter modules such as this are usually initialized such that the initial output of the adapter will always be zeros so as to prevent degradation of the original model's performance due to addition of such modules.

## Emulating a standard architecture

For this tutorial, the focus will be on demonstrating how to modify an existing architecture to add Adapter support.

We will focus on a trivial model implemented using simple Multi-Layer Perceptrons. Still, the model itself will emulate a standard Encoder-Decoder architecture (commonly used in multiple domains, such as ASR, NLP, NMT etc). 

We will also skip the implementation of datasets, data loaders, losses, metrics, and the Pytorch Lightning "steps" (trainer, validation, test). 

In [ ]:
import torch
import torch.nn as nn
from nemo.core import NeuralModule, ModelPT

from hydra.utils import instantiate
from omegaconf import DictConfig, OmegaConf

In [ ]:
class MLP(torch.nn.Module):
    def __init__(self, dim: int = 50):
        super().__init__()

        self.fc = torch.nn.Linear(dim, dim)
        self.ln = torch.nn.LayerNorm(dim)

    def forward(self, x):
        x = self.fc(x)
        x = self.ln(x)
        return x

class ResidualMLP(torch.nn.Module):
  def __init__(self, dim: int, num_layers: int):
    super().__init__()

    self.dim = dim
    self.num_layers = num_layers
    self.layers = nn.ModuleList([MLP(dim) for _ in range(num_layers)])
  
  def forward(self, x):
    input = x
    for layer in self.layers:
      x = layer(x)
      x = x + input
      input = x
    return x

-----
Next we implement a simple model that has two "modules"

In [ ]:
class SimpleModel(ModelPT):
    def __init__(self, cfg, trainer=None):
        super().__init__(cfg, trainer=trainer)

        self.encoder = instantiate(cfg.encoder)  # type: ResidualMLP
        self.decoder = instantiate(cfg.decoder)  # type: ResidualMLP
        self.projection = nn.Linear(self.decoder.dim, cfg.out_features)

    def forward(self, x):
        y = self.encoder(x)
        z = self.decoder(y)
        out = self.projection(z)
        return out

    def list_available_models(cls):
        return []

    def setup_training_data(self, train_data_config):
        pass

    def setup_validation_data(self, val_data_config):
        pass

## Initialize the basic model

The above model is a simple residual MLP network with two components, an encoder, and a decoder block. It may not do so well on real-world tasks, but it is sufficient for this demonstration.

Next, let's create a helper to generate a config for this model and create a new model using that config!

In [ ]:
def get_classpath(cls):
    return f'{cls.__module__}.{cls.__name__}'

def get_model_config(dim=512):
    config = OmegaConf.create(
        {
            'in_features': dim,
            'out_features': 10,
            'encoder': {'_target_': get_classpath(ResidualMLP), 'dim': dim, 'num_layers': 4},
            'decoder': {'_target_': get_classpath(ResidualMLP), 'dim': dim, 'num_layers': 2},
        }
    )
    return config

In [ ]:
dim = 512
model_cfg = get_model_config(dim)
model = SimpleModel(model_cfg)
model.summarize()

In [ ]:
# Check if the forward pass works !
with torch.no_grad():
  input_data = torch.randn(8, dim)
  out = model(input_data)
  print(out.shape)

# Incorporating Adapters - Module by Module

Now that we have a basic Model that we can successfully perform a forward pass on, we can add adapter support to the Model and its modules - layer by layer.

When considering the addition of adapter support, we work backward, going from the lowest level module used, and build a chain that forwards the methods of the adapters from the top-level Model to the bottom-level module(s) / layer(s).

# Adapter support in the lowest level module

As we work backward in the model chain, we look at the `MLP` module that creates a `Linear` and `LayerNorm` layer. We now extend this MLP module with the `AdapterModuleMixin` that is available inside `nemo.core.adapter_mixins`.

It is generally advised to directly update the code of the module, though there are other ways to implement this (shown later in the tutorial).

-----


## What is a `mixin`? 
A `mixin` is generally a term used to refer to a class that is **inherited by another class**, **adds some functionality to another class**, _but cannot be used on its own_. A mixin can be loosely considered a relatively safe way to incorporate additional functionality into a class via multiple inheritances. 

In [ ]:
from nemo.core import adapter_mixins

In [ ]:
help(adapter_mixins.AdapterModuleMixin)

In [ ]:
# NOTE: See the *two* classes being inherited here !
class MLP(torch.nn.Module, adapter_mixins.AdapterModuleMixin):
    def __init__(self, dim: int = 50):
        super().__init__()

        self.fc = torch.nn.Linear(dim, dim)
        self.ln = torch.nn.LayerNorm(dim)

    def forward(self, x):
        x = self.fc(x)
        x = self.ln(x)

        # The only necessary change to the module code !
        if self.is_adapter_available():
          x = self.forward_enabled_adapters(x)
        return x

    # add a utility method to calculate number of parameters (or we could simple extend nemo.core.NeuralModule instead)
    @property
    def num_weights(self):
      num: int = 0
      for p in self.parameters():
          if p.requires_grad:
              num += p.numel()
      return num

-----

That's it! We now have an MLP layer that has nearly full adapter support! We will try out a few of the adapter functionalities below to get a teaser of what we can expect as we go further into this tutorial

## Experimenting with a module level adapter

We will now instantiate the newly augmented `MLP` model above and explore all the functionality that has been added via the `AdapterModuleMixin` class - without having to write too much supporting code!

-----

First, let's create a `MLP` module and print the number of trainable parameters (before adding any adapters)

In [ ]:
mlp = MLP(dim)

print(mlp)
print("Num trainable parameters (without adapters):", mlp.num_weights)

## Adapter Modules

Next, let us import and add an adapter or two to this module! We first import `adapter_modules` from the NeMo `common` collections. This module contains pre-defined Adapter modules that can be attached to other torch.nn.Modules!

In [ ]:
from nemo.collections.common.parts import adapter_modules

In [ ]:
# Next we look at one of the adapter modules - the LinearAdapter
linear_adapter = adapter_modules.LinearAdapter(in_features=dim, dim=5)
print(linear_adapter)

-----
You will often not directly with this module, instead of passing the Config Dataclass to the `AdapterModuleMixin` methods. We see an example below - 

## [Optional] Constructing Adapter Components

Linear Adapter Modules are not the only type of adapters you can create ! In PyTorch, any torch.nn.Module can be made into an Adapter component.

For example, you can potentially convert a pre-existing pytorch module into an adapter component. The below section is **optional**, but is recommended if you wish to create your own adapters.


------
First, let us start with a simple PyTorch module.

In [ ]:
class SimpleModule(torch.nn.Module):
  def __init__(self, size: int):
    super().__init__()
    self.size = size
    self.model = torch.nn.Sequential(
        torch.nn.Linear(size, size, bias=False),
        torch.nn.Identity(),
    )
  
  def forward(self, x):
    return self.model(x)

### Adapter Strategy

Adapter modules are, at the end of the day, simply PyTorch modules. Just as PyTorch modules, they take some input tensors, perform some operation and then return some result.

There are many ways to integrate adapters - add them as a residual, multiply pointwise, concatenate with the input (at the end or the beginning). The Adapter Strategy class determines how an adapter integrates with its input.

In [ ]:
# The earlier LinearAdapter has a simple ResidualAddStrategy
# Uncomment below to see the ResidualAddAdapterStrategy definition
# help(linear_adapter.adapter_strategy)

### Creating a custom Adapter Strategy

Residual Add strategy can be considered the simple operation $f(x) = x + adapter(x)$ such that $adapter$'s initial outputs without training should be 0. 

In doing so, the output of the adapter augmented model is originally just $f(x) = x$, and so the model retains the exact performance of the original model (without any adapters).

-----

Below, we will create a Multiplication adapter strategy simply as a demonstration.

In [ ]:
from nemo.core.classes.mixins import adapter_mixin_strategies

We will implement a special `forward` method of adapters as follows

In [ ]:
# Uncomment to see the definition of the AbstractAdapterStrategy
# help(adapter_mixin_strategies.AbstractAdapterStrategy)

In [ ]:
class MultiplicationAdapterStrategy(adapter_mixin_strategies.AbstractAdapterStrategy):

  def __init__(self, scaling_factor: float = 1.0):
    super().__init__()
    self.scale = scaling_factor

  def forward(self, input: torch.Tensor, adapter: torch.nn.Module, *, module: 'AdapterModuleMixin'):
     # This is the forward method that takes in the previous input (here, its a tensor, but it can be a dictionary, a tuple, a class, anything really).
     # The second argument is the adapter that is currently being applied to this input
     # The final argument is the entire nn.Module that supports adapters.
     # In this case, the final argument would be the entire `MLP` module
     
     # Equivalent to f(x) = x * adapter(x)
     adapter_out = adapter(input)  # compute the adapter output from the input(s)
     result = input * adapter_out

     # Apply scaling factor. Equivalent to f(x) = scale * (x * adapter(x))
     result = self.scale * result
     return result


### Design a corresponding dataclass for the Adapter Strategy

In order to make usage of this class easier, you should create a Dataclass that can be used to create the strategy easily. We show an example below:

In [ ]:
from dataclasses import dataclass

@dataclass
class MultiplicationAdapterStrategyConfig:
    scaling_factor: float = 1.0

    # mandatory field
    _target_: str = "{0}.{1}".format(
        MultiplicationAdapterStrategy.__module__, MultiplicationAdapterStrategy.__name__
    )  

### Creating a Custom Adapter Component

Now that we have both the basic PyTorch module (`SimpleModule`) as well as the Adapter Strategy (`MultiplicationAdapterStrategy`), we can now construct a new adapter component.

The prime difference between a basic PyTorch module and an adapter component is the `adapter_strategy` - it defines how an adapter integrates with the original input. 

In [ ]:
class SimpleModuleAdapter(SimpleModule, adapter_modules.AdapterModuleUtil):

  def __init__(self, size: int, adapter_strategy: MultiplicationAdapterStrategy = None):
    """
    The input arguments should match the original module so you can pass the inputs to the module.
    It should also accept an adapter strategy.

    We will then use the method `setup_adapter_strategy()` to prepare the component to be used as an adapter.
    Note: Passing None to the strategy will let it pick a default strategy provided by the method
    `get_default_strategy_config()`.
    """
    super().__init__(size=size)

    # Prepare the adapter strategy
    self.setup_adapter_strategy(adapter_strategy)

    # Initialize the weights to be 0 at init
    self.reset_parameters()

  # Note: In this case, because we didn't add new modules, nor change how the original forward works
  # We dont need to subclass and override forward() !
  
  def reset_parameters(self):
    # We normally want an adapter at initialization to have no effect on the output
    # Therefore we replace the random uniform with a simple identity matrix, which will cause
    # the output of the adapter to match the input
    with torch.no_grad():
      self.model[0].weight = torch.nn.Parameter(torch.eye(self.size))
  

  def get_default_strategy_config(self) -> 'dataclass':
    """
    Make the default adapter strategy of this component be the `MultiplicationAdapterStrategy()`  
    """
    return MultiplicationAdapterStrategyConfig()

-----
Let's quickly test whether the adapter behaves as expected

In [ ]:
simple_adapter = SimpleModuleAdapter(size=5)
multiplication_strategy = simple_adapter.adapter_strategy
x = torch.randn(1, 5)
adapter_x = simple_adapter(x)
output = multiplication_strategy(input=x, adapter=simple_adapter, module=None)  # Normally you would pass the module here, but in this example can be skipped.
print("Original input :", x)
print("Adapter output :", adapter_x)
print("Strategy output:", output)

We see that the original input passes through the adapter, which results in the original values being returned successfully, and then the adapter strategy multiplies the two values (effectively computing the square of the input).

This is a sufficient demonstration of creating custom adapters, and we would normally not perform elementwise multiplication as an adapter strategy. Normally we would prefer the output of the strategy to be equal to the original init, at least at initialization.

### Design a corresponding dataclass for the Adapter Component

In order to make usage of this Adapter component easier, you should create a Dataclass that can be used to create the component easily. We show an example below:

In [ ]:
from typing import Optional

@dataclass
class SimpleModuleAdapterConfig:
    size: int
    adapter_strategy: Optional[MultiplicationAdapterStrategyConfig] = None

    # mandatory field
    _target_: str = "{0}.{1}".format(
        SimpleModuleAdapter.__module__, SimpleModuleAdapter.__name__
    )  

## Adding an Adapter Module

Since `MLP` inherits `AdapterModuleMixin`, it also inherits a set of methods that perform adapter module manipulations - such as adding a new adapter.

When users want to add an adapter, they can call `add_adapter()` with two specific arguments - `name` and `cfg`.

Arguments - 
- `name`: A string name that must be **locally unique** (for modules) and **globally unique** (for models). The name may also support ":" to delegate that the adapter belongs to specific modules only (this is discussed towards the end of the tutorial).
- `cfg`: A dataclass / OmegaConf config that contains the `_target_` attribute pointing to the classpath of an Adapter Module, along with any additional required attributes.

In [ ]:
mlp.add_adapter(name='adapter_1', cfg=adapter_modules.LinearAdapterConfig(in_features=dim, dim=5))

In [ ]:
# Now check the new parameter count of this MLP module, it should be higher than the previous count
print("New param count :", mlp.num_weights)

-----

**Note**: You can add as many adapters as are needed! While in this tutorial, we will only add one, we usually recommend adding one adapter for every task you want to specialize in. 

Also, note that while it is possible to train multiple adapters at once (add many adapters, enable them all, then unfreeze them), we recommend training just one adapter per task.

-----
**Note**: If you try to add the same adapter multiple times, you will see the below error message! 

Remember, adapter names must be **locally** unique at the module level and **globally** unique at the model level!

In [ ]:
# Uncomment to see the error message - 
# mlp.add_adapter(name='adapter_1', cfg=adapter_modules.LinearAdapterConfig(in_features=dim, dim=10))

## Get all enabled Adapter Modules

Next, we use `get_enabled_adapters()` to return a list of names of all the enabled adapters currently available to this module.

In [ ]:
mlp.get_enabled_adapters()

## Set the state of Adapter Modules

We can get the enabled adapter names with the above method, but how do we set whether an adapter module should be enabled or disabled? 

For that, we use the `set_enabled_adapter()` method. It has a few arguments - 
- `name`: An optional string name of an adapter, which will specifically enable or disable only that adapter. If no `name` is provided, all adapter modules will have their state set to the new value.
- `enabled`: A bool, whether the adapter should be enabled or not.

-----

Enabling an adapter simply enables the forward pass of that adapter and nothing more. It does not freeze / unfreeze the weights of the adapter itself, allowing more complex interactions to occur in combination with other adapters.

For example, one can add an adapter to a model, train it and then save the model. The restored model can then add yet another adapter. Prior to training this second adapter, the user can decide to utilize the outputs of the first adapter instead of the original model's outputs. To accomplish this, we can enable both adapters, but freezing the weights of the first adapter, and train just the second adapter.

In [ ]:
# Disable all adapters
mlp.set_enabled_adapters(enabled=False)
print("Enabled adapters :", mlp.get_enabled_adapters())

# Enable just one adapter
mlp.set_enabled_adapters(name="adapter_1", enabled=True)
print("Enabled adapters :", mlp.get_enabled_adapters())

## Check if Adapter Module(s) are available / enabled

An extension of the above two methods is to check if the current module has any active adapter module or not. To do so, you can use `is_adapter_available()`.


In [ ]:
mlp.is_adapter_available()

## Adapter functionality methods

The above few methods form the core of the functionality to enable adapters to be added and modified to a module, but they don't use the added adapter modules!

Therefore, the following functionality methods are used to leverage adapters properly and need not be overridden by the user (unless required for some special case).

### `forward_enabled_adapters()`
To use these adapters, we utilize the `forward_adapter_modules()` method.

To utilize any enabled adapters, the module that inherits `AdapterModuleMixin` should first check if any adapters are enabled and then call this method to forward the adapter modules on the input data. 


In [ ]:
# Check `forward_enabled_adapters()`
out = mlp.forward_enabled_adapters(input_data)
print(out.shape)

### `forward_single_enabled_adapter_()`
A method that can be sub-classed in order to provide custom logic for the forward pass of the adapters. For example, we may wish to provide some adapters with different set of inputs, or check whether we support an adapter type or not before we perform forward pass.

It can be useful to check the type of the adapter, and then use the additional information prior to forwarding the input to any specific adapter.

In [ ]:
# Check `forward_single_enabled_adapter_()`
adapter_name = mlp.get_enabled_adapters()[0]  # we have enabled just one adapter
adapter_module = mlp.adapter_layer[adapter_name]  # get the adapter module with this name
adapter_strategy = adapter_module.adapter_strategy  # get the adapter strategy for this adapter

out = mlp.forward_single_enabled_adapter_(input_data, adapter_module, adapter_name=adapter_name, adapter_strategy=adapter_strategy)
print(out.shape)

-----
For further information about adapter forward pass, adapter strategy please refer to the documentation section for adapters.

### `unfreeze_enabled_adapters()`
One of the core benefits of adapters is that they do not need the entire model to be trained. We can freeze the rest of the original model/modules and train the adapter modules themselves. 

We can do this in two steps - 
- Call model.freeze() (at the highest level)
- Call `unfreeze_enabled_adapters()` that will recursively unfreeze just the adapter modules that are enabled.

In [ ]:
# First setup some utility functions (this is part of NeuralModule)
def freeze(m):
    for param in m.parameters():
      param.requires_grad = False
    m.eval()

def unfreeze(m):
    for param in m.parameters():
      param.requires_grad = True
    m.train()

In [ ]:
freeze(mlp)
print("MLP frozen params :", mlp.num_weights)

In [ ]:
# Check `unfreeze_enabled_adapters()` - param count should be lower than the previous total (original + adapter)
mlp.unfreeze_enabled_adapters()
print("MLP unfrozen adapter params :", mlp.num_weights)

# Adapter support in intermediate level modules

Above, we discussed many of the methods and capabilities added to a simple nn.Module via the `AdapterModuleMixin`. However, this module was the lowest building block in the model. Next, we will look into how to "dispatch" the calls from the intermediate module to the lower modules.

We will aim for simplicity in this tutorial, modifying the minimal amount of code as possible. However, it is entirely possible to add much more sophisticated handling of intermediate layer dispatches to lower level modules.

## Intermediate modules that are instantiated via config

Currently, we have a 3 level model -- 

`Top level Model (SimpleModel) -> Intermediate level Module (ResidualMLP) -> Bottom level Module (MLP)`. 

-----

As you may have noticed, in earlier primer tutorials (NeMo Model Primer), we recommend the Model utilize configs to instantiate its intermediate modules. This allows users to swap in equivalent modules via the config and enjoy the rest of the utility of the Model itself without too many code changes.

For such "penultimate" modules, we recommend creating a separate Adapter supported module that extends the original module rather than modifying the original module itself. This is merely a preference to avoid cluttering the original module code and can be ignored if the user wishes.

For this guide, we will show the recommended setup so that best practices can be followed.

## Creating an Adapter-compatible "Penultimate" module

First, we create the new Adapter compatible module as a separate class.

In [ ]:
# NOTE: We subclass the original ResidualMLP, and add in the AdapterModuleMixin too
class ResidualMLPAdapter(ResidualMLP, adapter_mixins.AdapterModuleMixin):
  pass

## Overriding the adapter methods

Next, we override a few adapter methods, such that we dispatch these methods to all the blocks of `MLP` inside of the `ResidualMLP` module.

Therefore, this will create/update the state / forward an adapter module inside the `MLP` modules!

In [ ]:
from typing import List, Optional

class ResidualMLPAdapter(ResidualMLP, adapter_mixins.AdapterModuleMixin):
  def add_adapter(self, name: str, cfg: DictConfig):
      # call the same method on each `MLP` layer, collecting results
      for layer in self.layers:
        layer.add_adapter(name, cfg)
      
  def get_enabled_adapters(self) -> List[str]:
      # call the same method on each `MLP` layer, collecting results
      enabled_adapters = set([])
      for layer in self.layers:
        names = layer.get_enabled_adapters()
        enabled_adapters.update(names)
      return list(enabled_adapters)
  
  def set_enabled_adapters(self, name: Optional[str], enabled: bool):
      # call the same method on each `MLP` layer, collecting results
      for layer in self.layers:
        layer.set_enabled_adapters(name, enabled)
  
  def is_adapter_available(self) -> bool:
      # call the same method on each `MLP` layer, collecting results
      is_available = any([layer.is_adapter_available() for layer in self.layers])
      return is_available

## Register the new adapter

When we subclass a module to add Adapter functionality, it is essential to register such modules with the Adapter registry so that many convenient functions can be used later on. The adapter registry is a global collection of the base class and adapter compatible class that can be used later to update model configs more easily.

The steps below are : 
- Check if the registry has the base class via `get_registered_adapter()`.
- If it returns None, then register the base class and its compatible adapter class via `register_adapter()`.

-----

**Note**: that while in this trivial case, our penultimate module is, in fact the intermediate module, there may be real-world models with many more intermediate modules. In such a case, you may update such intermediate modules by directly extending `AdapterModuleMixin` and following the above steps without creating a new subclass. In such cases, you can also skip registering for these modules.

In [ ]:
if adapter_mixins.get_registered_adapter(ResidualMLP) is None:
  adapter_mixins.register_adapter(ResidualMLP, ResidualMLPAdapter)

-----

That's all it takes to add support for intermediate modules! While adding the same (or similar) code for all intermediate modules may seem a little redundant, that is only because we are implementing the most naive dispatching. 

There are many interesting approaches to building adapters, such as adapters for only attention layers (before or after) or only for the final feed-forward (in conventional attention-based blocks). As such, intermediate layers have the total flexibility to dispatch these functions to lower layers.

# Adapters support at top level Model

Finally, after dispatching the above methods from the intermediate modules to the bottom module, we need to perform the final dispatch from the Model itself to the first (or penultimate if moving backward) module.

In this case, we will subclass a different mixin class than the one we have been using till now. Instead of `AdapterModuleMixin`, we will instead subclass `AdapterModelPTMixin` - which has some functionality built into it to manage model level config (including saving and restoring adapter compatible models !)



## Extending `AdapterModelPTMixin`

There are two ways in which one can inherit the top level mixin - 

(1)  directly extend it in your current Model class

(2)  create a class that implements the additional functionality, then inherit that class.

It might seem that option (2) is a more roundabout way of achieving (1). Still, it is done to keep the logic of adapter management outside of the complicated Model codebase since the Model itself is involved with many important details such as the setup of the modules, data loaders, optimizer/schedulers, losses, metrics, and the Pytorch Lightning "steps" - training, validation, and testing steps.

-----
We will follow option (2) in this tutorial for clarity. Also, note that we will create new subclasses for each step for transparency (and to avoid burdening you with too much information at once). It is preferred to do all these steps within just one new class.

In [ ]:
class SimpleModelAdapter(adapter_mixins.AdapterModelPTMixin):
  pass

In [ ]:
help(adapter_mixins.AdapterModelPTMixin)

## Overriding methods for selective dispatch

There was an apparent distinction between how to dispatch adapter calls to the subsequent module. This was because the modules were homogeneous and shared typical behavior.

However, there is no reason for the subsequent layers to share standard behavior at the model level. Think in terms of `encoder` vs. `decoder` Transformer layers - they are significantly different modules! Therefore why should their adapters be similar?

At the top level, we can utilize input from the user to determine how to construct adapters for such logically heterogeneous components. The following sections will describe how we can utilize **global** and **module** level adapters to separate the behavior and construction of adapters for each **component** of the Model.

## Overriding setup_adapters()

When a model must be restored, it must carefully load the parameters of all the modules within it. Till now, we have been able to augment torch.nn.Module(s) with Adapter information and capabilities, but we have not preserved this information anywhere.

Therefore if we were to turn off the notebook and try to restore a saved checkpoint, restoration would fail because the new model does not have the information of the adapters that it previously added, and so state dict matching will fail.

This issue is resolved by overriding `setup_adapters()` and calling it inside the Model constructor.

In [ ]:
# Import the class explicitly to make instance checks easier
from nemo.core.classes.mixins.adapter_mixins import AdapterModuleMixin

class SimpleModelAdapterSetupAdapters(SimpleModelAdapter):
  def setup_adapters(self):
    # First check that any of the modules support adapters or not
    supports_adapters = False

    # Check the inheriting class' modules supports adapters or not
    if hasattr(self, 'encoder') and isinstance(self.encoder, AdapterModuleMixin):
        supports_adapters |= True

    if hasattr(self, 'decoder') and isinstance(self.decoder, AdapterModuleMixin):
        supports_adapters |= True

    # If any class supports it, try to restore adapters
    if supports_adapters:
        super().setup_adapters()

-----

In this step, we merely check if any of the modules we have created support adapters or not. If any of them do, we call the super() method to try to restore any adapters if needed.

## Overriding add_adapter()

Next up, we override `add_adapter`. Before jumping to the code, we must first discuss the types of adapters supported in NeMo.

- `Global Adapters`: These adapters share their name and functionality with all supported modules. They are helpful when you are sure that one adapter can be shared between multiple model components. For example, the encoder and decoder share the same adapter.
- `Module Adapters`: These adapters are specific to each module they are designated to and, therefore, cannot share their name across multiple components of a model. They are denoted via the adapter name of the format `{module_name}:{adapter_name}`.

**Note**: After the module adapter is added, it can be referred to simply by the `adapter_name` part of its name. There is no need to provide `module_name` again since it is guaranteed that all adapter names are globally unique at the Model level.

-----

It is entirely up to the user whether they should support just `Global Adapters`, `Module Adapters` or both. For the purpose of this tutorial, we will support both, and also add support for a `Default Module Adapter` for the `encoder`.

**Note**: In order to easily distinguish between a `Global` and `Module` adapter, use the convenient method `resolve_adapter_module_name_(name)`. We encourage use of the property `adapter_module_names` to determine the valid adapter modules that can be used.

In [ ]:
class SimpleModelAdapterAddAdapter(SimpleModelAdapterSetupAdapters):

  def add_adapter(self, name: str, cfg: DictConfig):
      # Setup the config first. At the model level, super does not automatically call any of the subclass methods
      # It just sets up the model.cfg for users
      super().add_adapter(name, cfg)

      # Resolve module name and adapter name
      module_name, adapter_name = self.resolve_adapter_module_name_(name)

      # Try to retrieve global adapter config
      global_config = self._get_global_cfg()

      # forward the method call to the individual modules
      # If module name is empty, it is a default and global adapter, otherwise it is a module adapter
      if (module_name == '' and global_config.get('encoder_adapter', True)) or (module_name == 'encoder'):
          self.encoder.add_adapter(name, cfg)

      if (module_name == '' and global_config.get('decoder_adapter', False)) or (module_name == 'decoder'):
          self.decoder.add_adapter(name, cfg)
    
  def resolve_adapter_module_name_(self, name: str) -> (str, str):
      # resolve name and module
      module_name, adapter_name = super().resolve_adapter_module_name_(name)

      # '' as module name means "default module"
      # assert that the module name (if provided) is valid - default, encoder or decoder
      valid_module_names = self.adapter_module_names  # Get the list of supported adapter modules from property
      if module_name not in valid_module_names:
          raise ValueError(f"Provided module name `{module_name}` is not in valid list : {valid_module_names}")

      return (module_name, adapter_name)

  def _get_global_cfg(self):
      # Utility method to get a default "global" adapter config (can be given any value by the user in this config)
      global_config = DictConfig({})
      if 'adapters' in self.cfg and self.adapter_global_cfg_key in self.cfg.adapters:
          global_config = self.adapter_cfg[self.adapter_global_cfg_key]
      return global_config

  @property
  def adapter_module_names(self) -> List[str]:
      module_names = super().adapter_module_names  # "Default" adapter module: ''
      module_names.extend(['encoder', 'decoder'])  # Add support for `encoder` and `decoder` modules
      return module_names


-----

Since that was a lot of code, let's break it down. First we define the utility method `_get_global_cfg()` that tries to get the "model.cfg.adapters.global_cfg" sub-config from the model config. This config is user-defined and can be used by the user to configure any logic as necessary. If it is not found, a default dict is created instead.

Next, we override the `resolve_adapter_module_name_(name)` method. This method in the base class takes a string name and tries to split it into `module_name` and `adapter_name`. We override this method to assert some valid `module_name's.

-----

Finally, we override `add_adapter` - first, we call super() to update the config. Next, we call our overridden `resolve_adapter_module_name_(name)` to check if the provided adapter name is valid or not. Then we gather the Adapter `global_cfg` if it exists in the model config.

With all this information, we are now ready to add adapters as needed. We have some "user-defined" logic, such that we add an encoder adapter if either. 
- The user provides a "Global" adapter with a default module name, OR has set the value of `global_cfg.encoder_adapter` to True (True by default). This means that at least the encoder adapter will always be added by default.
- The user provides a `Module` adapter with the `decoder` module name, OR has set the `global_cfg.decoder_adapter` flag to True explicitly (False by default).

## Overriding get_enabled_adapters()

Next, we will override the `get_enabled_adapters()` method. This is often simple enough, where we need only to check if the Model components support adapters or not, and then if they do, gather and collate the results of those nodules.

In [ ]:
class SimpleModelAdapterGetEnabledAdapters(SimpleModelAdapterAddAdapter):

  def get_enabled_adapters(self) -> List[str]:
      enabled_adapters = super().get_enabled_adapters()

      # Forward the method call to the individual modules
      if isinstance(self.encoder, AdapterModuleMixin):
          encoder_adapters = self.encoder.get_enabled_adapters()
          enabled_adapters.extend(encoder_adapters)

      if isinstance(self.decoder, AdapterModuleMixin):
          decoder_adapters = self.decoder.get_enabled_adapters()
          enabled_adapters.extend(decoder_adapters)

      return enabled_adapters

## Overriding set_enabled_adapters()

Similar to the above, we only need to check if the components support adapters or not and then dispatch the call to those components if they are supported.

**Note**: We will perform a logic check here instead of the usual inheritance check. An inheritance check alone does not mean the component has added an adapter or not - remember, we have two use cases of `default/global/module encoder` and `module decoder` adapters. So we need to check for those conditions.

**Note 2**: As you may remember, set_enabled_adapters() does take the value `None` for the name to set the state of all adapters. However, `resolve_adapter_module_name(name)_` must always accept a valid string name. So care must be taken not to pass `None` to that method.

In [ ]:
class SimpleModelAdapterSetEnabledAdapters(SimpleModelAdapterGetEnabledAdapters):

  def set_enabled_adapters(self, name: Optional[str] = None, enabled: bool = True):
      # check if valid model with some adapter support
      super().set_enabled_adapters(name, enabled)

      # Resolve module name and adapter name
      if name is not None:
          module_name, _ = self.resolve_adapter_module_name_(name)
      else:
          module_name = None

      # Try to retrieve global adapter config
      global_config = self._get_global_cfg()

      # Forward the method call to the individual modules
      # Note the OR checks - 
      # if module_name is None - ie explicitly None was passed, set the state for all modules
      # if module name was '' or 'encoder, or if `global_cfg.encoder_adapter` was true, or module_name was '' or 'encoder', forward to encoder.
      # if `global_cfg.decoder_adapter` was true, or module_name was 'decoder', forward to decoder.
      # The user can chose to simplify this logic, or add more complex logic as required.
      if name is None or global_config.get('encoder_adapter', True) or module_name in ('', 'encoder'):
        if self.encoder.is_adapter_available():
          self.encoder.set_enabled_adapters(name, enabled)

      if name is None or global_config.get('decoder_adapter', False) or module_name == 'decoder':
        if self.decoder.is_adapter_available():
          self.decoder.set_enabled_adapters(name, enabled)

## Overriding `check_valid_model_with_adapter_support_()`

In the above implementation, we implicitly check that the components of the model support adapters or not and carry forward. This is fine, but we may want to perform more strict checks so as to give meaningful warnings or errors about invalid combinations of actions. 

For this purpose, we provide `check_valid_model_with_adapter_support()_`. This method is called before nearly all adapter operations, and attempts to assert some truths. Users can raise any error or warning here to notify the user about invalid operations / configurations.

In [ ]:
from nemo.utils import logging, logging_mode

class SimpleModelAdapterFinal(SimpleModelAdapterSetEnabledAdapters):

  def check_valid_model_with_adapter_support_(self):
      global_cfg = DictConfig({})
      if self.adapter_global_cfg_key in self.adapter_cfg:
          global_cfg = self.adapter_cfg[self.adapter_global_cfg_key]

      encoder_adapter = global_cfg.get('encoder_adapter', True)
      decoder_adapter = global_cfg.get('decoder_adapter', False)

      if encoder_adapter and not hasattr(self, 'encoder'):
          logging.warning("Encoder not available", mode=logging_mode.ONCE)
      elif encoder_adapter and not isinstance(self.encoder, AdapterModuleMixin):
          logging.warning("Encoder does not support adapters !", mode=logging_mode.ONCE)

      if decoder_adapter and not hasattr(self, 'decoder'):
          logging.warning("Decoder is not available", mode=logging_mode.ONCE)
      elif decoder_adapter and not isinstance(self.decoder, AdapterModuleMixin):
          logging.warning("Decoder does not support adapters !", mode=logging_mode.ONCE)

## Updating the Model

After the top-level Model mixin class has been implemented separately, we can easily attach it to our original Model. For the tutorial's sake, we will duplicate the code here, but know that you can also subclass the Model and override its `__init__` method for similar functionality.

In [ ]:
# Note how we added `SimpleModelAdapterFinal` to the class inheritance scheme.
# The only other change is the addition of `self.setup_adapters()` to the __init__ method.
class SimpleModel(ModelPT, SimpleModelAdapterFinal):
    def __init__(self, cfg, trainer=None):
        super().__init__(cfg, trainer=trainer)

        self.encoder = instantiate(cfg.encoder)  # type: ResidualMLP
        self.decoder = instantiate(cfg.decoder)  # type: ResidualMLP
        self.projection = nn.Linear(self.decoder.dim, cfg.out_features)

        # NOTE: The only important change - calling `setup_adapters()` !
        self.setup_adapters()

    def forward(self, x):
        y = self.encoder(x)
        z = self.decoder(y)
        out = self.projection(z)
        return out

    def list_available_models(cls):
        return []

    def setup_training_data(self, train_data_config):
        pass

    def setup_validation_data(self, val_data_config):
        pass

-----

And there we go ! Just subclassing the new mixin, as well as calling `setup_adapters()` is the only modification necessary !

In [ ]:
old_config = get_model_config(dim)
model = SimpleModel(old_config)
model.summarize()

-----

Now, let us try adding a `decoder` Module adapter to this Model.

In [ ]:
# This cell will error out if uncommented
# model.add_adapter("decoder:adapter_1", cfg=adapter_modules.LinearAdapterConfig(in_features=dim, dim=5))

-----

It fails with an error `Encoder does not support adapters !`. This is because, if you recall, the original config of this model (old_config) has the classpath to the `ResidualMLP` class, but not the `ResidualMLPAdapter` class!

This can be easily rectified because we have already registered this class properly (refer to `Register the new adapter` sub-section)

In [ ]:
def get_adapter_model_config() -> DictConfig:
  config = get_model_config()

  # Find the metadata in the registry, and get the correct adapter capable class path
  enc_adapter_metadata = adapter_mixins.get_registered_adapter(config.encoder._target_)
  if enc_adapter_metadata is not None:
      print("Updated encoder to support adapters !")
      config.encoder._target_ = enc_adapter_metadata.adapter_class_path

  # Find the metadata in the registry, and get the correct adapter capable class path
  dec_adapter_metadata = adapter_mixins.get_registered_adapter(config.decoder._target_)
  if dec_adapter_metadata is not None:
      print("Updated decoder to support adapters !")
      config.decoder._target_ = dec_adapter_metadata.adapter_class_path

  return config

In [ ]:
new_config = get_adapter_model_config()
model = SimpleModel(new_config)
model.summarize()

-----
Now let us again try to add a `decoder` Module Adapter -

In [ ]:
model.add_adapter('decoder:adapter_1', cfg=adapter_modules.LinearAdapterConfig(in_features=dim, dim=5))

-----
You will see multiple log messages stating `adapter_1` was added (one for each block). Now lets check if the adapter is in the correct module - 

In [ ]:
print("Encoder adapter available :", model.encoder.is_adapter_available())
print("Decoder adapter available :", model.decoder.is_adapter_available())
print("Decoder adapter(s) :", model.decoder.get_enabled_adapters())

In [ ]:
model.summarize()

## Preparing to train an adapter

Finally, now that our model has adapter capabilities, we can train just the adapter while freezing the rest of the model.

In the following section, we show how to perform this setup.

In [ ]:
# disable all adapters, enable just one adapter that we want to train
model.set_enabled_adapters(enabled=False)
model.set_enabled_adapters('adapter_1', enabled=True)  # note : we directly use the adapter_name of adapter_1

# freeze all the weights, unfreeze just the enabled adapters
model.freeze()
model.unfreeze_enabled_adapters()

print()
model.summarize()

## Save and Restore Adapters

Now that we have implemented this Model, we can always use `model.save_to()` to save a NeMo model with full adapter support. Let us try to save and then restore this adapter model.

In [ ]:
model.save_to('full_model.nemo')

In [ ]:
!ls -d -- *.nemo

In [ ]:
new_model = ModelPT.restore_from('full_model.nemo')

In [ ]:
new_model.decoder.get_enabled_adapters()

-----

While we can save and restore the entire model, we don't need to. Think about adapters for a moment - they are additional modules on top of a base model. For every new adapter we add, it would not be feasible to save and restore the entire model to file (especially if the models are multiple billions of parameters !).

Next, we discuss how to save and restore just the modules themselves into separate .pt files using `save_adapters()`.

In [ ]:
model.save_adapters('adapters.pt', name=None)

In [ ]:
!du -sh adapters.pt full_model.nemo

-----
As you can see, the whole model is much larger than just the adapter modules themselves. This can be used to share just the adapter modules with others without using ample disk space for the entire model.

Next, we show how to restore such adapter checkpoint to a new model using `load_adapters()`.

In [ ]:
new_config = get_adapter_model_config()
model_2 = SimpleModel(new_config)
model_2.summarize()  # no adapters in basic model with adapter support

In [ ]:
model_2.load_adapters('adapters.pt', name=None, map_location='cpu')

In [ ]:
model_2.freeze()
model_2.unfreeze_enabled_adapters()
model_2.summarize()

In [ ]:
model_2.get_enabled_adapters()

-----
You will note that we passed None to the above model, restoring all the adapters, but what if we want to restore just one adapter from the checkpoint? Then we can pass the `name` argument to the `load_adapters()` method. 

Remember: The name passed here must be the name used to create the adapter itself, so for module-level adapters, the `module_name` must also be provided.

# Further reading

For further information about Adapters, please refer to the NeMo Documentation page for Adapters. It also includes sections that detail how to create your Adapter modules.

For further details regarding how adapters can be used for, please refer to the following articles - 
- [Parameter-Efficient Transfer Learning for NLP](https://arxiv.org/abs/1902.00751)
- [Exploiting Adapters for Cross-lingual Low-resource Speech Recognition](https://arxiv.org/abs/2105.11905)
- [Efficient Adapter Transfer of Self-Supervised Speech Models for Automatic Speech Recognition](https://arxiv.org/abs/2202.03218)